In [1]:
import tensorflow as tf
import pickle
import os
import shutil
import tempfile
import datetime
from glob import iglob
import time
# import attention
from collections import deque
import pickle
import pyreader
import numpy as np
import sys
import csv
from batchmake import Batcher
import astwalker
import ast
import tokenize
from io import StringIO

In [2]:
pad_token, pad_id = "§PAD§", 0
oov_token, oov_id = "§OOV§", 1
indent_token = "§<indent>§"
dedent_token = "§<dedent>§"
number_token = "§NUM§"


In [3]:
datas= 'data_samples/'
word_to_id='data_samples/mapping.map'
with open(word_to_id, 'rb') as f:
    word_to_id = pickle.load(f)
inv_map = {v:k for k, v in word_to_id.items() }
batch_data = datas+'output.txt.part0'
# batch_data = datas+'preprocess.part0'
with open(batch_data, 'rb') as f:
    picked_data = pickle.load(f)
    
print(len(picked_data))
train_proportion = 0.5
valid_proportion = 0.2
word_to_id[number_token]

481


12

In [44]:
word_to_id['.'], word_to_id['\n']
word_to_id

{'var3817': 5989,
 'function1047': 13800,
 'var3164': 1359,
 'attribute714': 11670,
 'attribute2385': 12157,
 'Class90': 818,
 'arg2217': 9431,
 'attribute2336': 12158,
 'arg1555': 9432,
 "'comment'": 1048,
 'var3846': 4704,
 'function2262': 12160,
 'arg1769': 12161,
 'arg1442': 1216,
 'var1471': 1739,
 'arg1581': 7545,
 'function145': 4705,
 'attribute1123': 349,
 'zipfile': 4707,
 'function1816': 8311,
 'attribute1035': 9433,
 'arg1419': 7546,
 'attribute1666': 12949,
 'attribute1121': 12867,
 'files': 668,
 'var643': 4708,
 'function1294': 8312,
 'var677': 8313,
 'attribute1691': 12165,
 'var814': 13858,
 "'40'": 5275,
 'function1774': 4119,
 'arg2017': 12542,
 'arg392': 9435,
 'function1501': 12167,
 'function306': 6721,
 'function2640': 9436,
 'var2487': 4709,
 'BeautifulSoup': 260,
 'function2461': 8314,
 'attribute1186': 10757,
 "'del'": 4710,
 'var985': 6708,
 'attribute270': 9437,
 'chr': 372,
 'attribute849': 9438,
 'var2481': 487,
 'arg1431': 7547,
 'var118': 1740,
 'functio

In [32]:
vocabulary = 'vocab.csv'
vocabs = []
with open(vocabulary, 'r', newline='', encoding='utf-8') as vocab:
    words = csv.reader(vocab)
    for word in words:
        vocabs.append(word)
print(len(vocabs))

3796


In [45]:
vocabs = [val for i, val in enumerate(word_to_id)]
vocabs

['var3817',
 'function1047',
 'var3164',
 'attribute714',
 'attribute2385',
 'Class90',
 'arg2217',
 'attribute2336',
 'arg1555',
 "'comment'",
 'var3846',
 'function2262',
 'arg1769',
 'arg1442',
 'var1471',
 'arg1581',
 'function145',
 'attribute1123',
 'zipfile',
 'function1816',
 'attribute1035',
 'arg1419',
 'attribute1666',
 'attribute1121',
 'files',
 'var643',
 'function1294',
 'var677',
 'attribute1691',
 'var814',
 "'40'",
 'function1774',
 'arg2017',
 'arg392',
 'function1501',
 'function306',
 'function2640',
 'var2487',
 'BeautifulSoup',
 'function2461',
 'attribute1186',
 "'del'",
 'var985',
 'attribute270',
 'chr',
 'attribute849',
 'var2481',
 'arg1431',
 'var118',
 'function2402',
 'Class93',
 'function59',
 'function626',
 'arg1588',
 'attribute2016',
 'function1452',
 'function1958',
 'var3511',
 'arg1744',
 'attribute195',
 'function1692',
 'attribute1167',
 'var4491',
 'arg1653',
 'var2362',
 'var3926',
 'release',
 'var2988',
 'function2232',
 'attribute1157',
 'f

In [48]:
word_to_id = {i:w for w,i in enumerate(vocabs)}

inv_map = {w:i for w,i in enumerate(vocabs)}
word_to_id['.'], inv_map[1306]

(1306, '.')

In [4]:
subdirectories = [os.path.join(datas,o) for o in os.listdir(datas) if os.path.isdir(os.path.join(datas,o))]

In [5]:
python_files = [(directory, [y for x in os.walk(directory) for y in iglob(os.path.join(x[0], '*.py'))])
                for directory in subdirectories]

In [6]:
train_split = int(len(python_files) * train_proportion)
valid_split = train_split + int(len(python_files) * valid_proportion)
train_files = []
valid_files = []
test_files = []

In [7]:
for project in python_files[:train_split]:
    train_files.extend([f[len(datas):] for f in project[1]])
for project in python_files[train_split:valid_split]:
    valid_files.extend([f[len(datas):] for f in project[1]])
for project in python_files[valid_split:]:
    test_files.extend([f[len(datas):] for f in project[1]])

In [8]:
def write_to_file(fname, lst):
    with open(os.path.join(datas, fname), "w") as write_file:
        for f in lst:
            print(f, file=write_file)

In [9]:
write_to_file("train_files.txt", train_files)
write_to_file("valid_files.txt", valid_files)
write_to_file("test_files.txt", test_files)

### 데이터셋을 train, valid, test셋으로 만든다.

### 파일 읽기

In [10]:
python_files = [os.path.join(datas, f) for f in train_files]
mapping = (lambda x: x) if word_to_id is None else (lambda x: word_to_id.get(x, oov_id))
def get_source_tree(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        fstr = f.read()
    fstr = fstr.replace('\r\n', '\n').replace('\r', '\n')
    if not fstr.endswith('\n'):
        fstr += '\n'
    return fstr, ast.parse(fstr, filename=filename)

In [11]:
def preprocess(tokentype, tokenval):
    if tokentype == tokenize.NUMBER:
        return number_token

    elif tokentype == tokenize.INDENT:
        return indent_token

    elif tokentype == tokenize.DEDENT:
        return dedent_token

    return tokenval

In [12]:
data = []
definition_positions = []
identifier_usage = []
gen_def_positions = True
error_data = 0
data_file_name = []
for filename in python_files:
    try:
        source, tree = get_source_tree(filename)
        tokens = tokenize.generate_tokens(StringIO(source).readline)
        data.append([(mapping(preprocess(tokenType, tokenVal)), start) for tokenType, tokenVal, start, _, _
                     in tokens
                     if tokenType != tokenize.COMMENT and
                     not tokenVal.startswith("'''") and
                     not tokenVal.startswith('"""') and 
                     (tokenType == tokenize.DEDENT or tokenVal != "")])
        data_file_name.append(filename)
        if gen_def_positions:
            walker = astwalker.ASTWalker()
            walker.walk(tree)
            definition_positions.append(walker.definition_positions)
            identifier_usage.append(walker.name_usage)
    except:
        error_data+=1
        continue
#         print("Error when tokenizing %s: %s" % (filename, sys.exc_info()[:3]))

### 전체 파일 ( python_files ) 중에서 에러난거 뺀데이터 (data) 

In [13]:
len(python_files),error_data,len(data), len(definition_positions)

(31, 4, 27, 27)

### 데이터 비교

In [14]:
dd,d=get_source_tree(data_file_name[1])
print(dd)

import sys
import os
sys.path.insert(0, os.path.abspath('..'))
var3213 = ['sphinx.ext.autodoc', 'sphinx.ext.viewcode']
var702 = ['_templates']
var519 = '.rst'
var2786 = 'index'
var2133 = 'zhihu-py3'
var389 = '2015, 7sDream'
var1450 = '0.1'
var1326 = '0.1'
var2147 = ['_build']
var2401 = 'sphinx'
var2501 = (os.environ.get('READTHEDOCS', None) == 'True')
if (not var2501):
    import sphinx_rtd_theme
    var2506 = 'sphinx_rtd_theme'
    var722 = [sphinx_rtd_theme.get_html_theme_path()]
else:
    var2506 = 'default'
var4586 = ['_static']
var1086 = 'zhihu-py3doc'
var1323 = {}
var56 = [('index', 'zhihu-py3.tex', 'zhihu-py3 Documentation', '7sDream', 'manual')]
var1033 = [('index', 'zhihu-py3', 'zhihu-py3 Documentation', ['7sDream'], 1)]
var1893 = [('index', 'zhihu-py3', 'zhihu-py3 Documentation', '7sDream', 'zhihu-py3', 'One line description of project.', 'Miscellaneous')]



In [15]:
token_to_str = []
for tokend_data,position in data[1]:
     token_to_str.append(inv_map[tokend_data])
print(' '.join(token_to_str))

import sys 
 import os 
 sys . path . §OOV§ ( §NUM§ , os . path . abspath ( '..' ) ) 
 var3213 = [ §OOV§ , §OOV§ ] 
 var702 = [ §OOV§ ] 
 var519 = §OOV§ 
 var2786 = 'index' 
 var2133 = 'zhihu-py3' 
 var389 = §OOV§ 
 var1450 = §OOV§ 
 var1326 = §OOV§ 
 var2147 = [ §OOV§ ] 
 var2401 = §OOV§ 
 var2501 = ( os . environ . get ( §OOV§ , None ) == 'True' ) 
 if ( not var2501 ) : 
 §<indent>§ import §OOV§ 
 var2506 = §OOV§ 
 var722 = [ §OOV§ . §OOV§ ( ) ] 
 §<dedent>§ else : 
 §<indent>§ var2506 = 'default' 
 §<dedent>§ var4586 = [ §OOV§ ] 
 var1086 = §OOV§ 
 var1323 = { } 
 var56 = [ ( 'index' , §OOV§ , §OOV§ , '7sDream' , §OOV§ ) ] 
 var1033 = [ ( 'index' , 'zhihu-py3' , §OOV§ , [ '7sDream' ] , §NUM§ ) ] 
 var1893 = [ ( 'index' , 'zhihu-py3' , §OOV§ , '7sDream' , 'zhihu-py3' , §OOV§ , §OOV§ ) ] 



### 원본데이터와 토큰데이터

In [16]:
type_max_rands = {
    "var": 4750, "function": 2900, "Class": 440, "attribute": 2400, "arg": 2400
}

identifier_types = [key for key in type_max_rands]
def_positions = [[[t[1] for t in fp if t[0] == k] for k in identifier_types] for fp in definition_positions]
# ex) def_positions[101] #function, var, class, attribute, arg

In [17]:
inputs = []
line_data_list = []
for line_data in data:
    for i, token in enumerate(line_data):
#         print(token[0])
#         inputs.append(token[0])
        line_data_list.append(token[0])
#         inputs.append(line_data_list)
#         line_data_list=[]
            
#             print(token[0])
        

In [18]:
# for row in line_data_list:
#     print(row)
print(len(line_data_list))

38661


In [19]:
def make_dataset(sequence, labels):
    ex = tf.train.SequenceExample()
    sequence_length = len(sequence)
    ex.context.feature['length'].int64_list.value.append(sequence_length)
    fl_tokens = ex.feature_lists.feature_list['tokens']
    fl_labels = ex.feature_lists.feature_list['labels']
    for token, label in zip(sequence, labels):
        fl_tokens.feature.add().int64_list.value.append(token)
        fl_labels.feature.add().int64_list.value.append(label)
    return ex

In [20]:
sequence_length=5

token = line_data_list[:sequence_length]
label = line_data_list[1:sequence_length+1]
ex = make_dataset(token,label)

In [21]:
# with open('inputs.csv', 'w',newline='', encoding='utf-8') as csvfile:
#     spamwriter = csv.writer(csvfile)
#     for row_line in inputs:
#         spamwriter.writerow(row_line)
# with open('inputs.csv', 'w',newline='', encoding='utf-8') as csvfile:
#     spamwriter = csv.writer(csvfile)
#     for row_line in inputs:
#     spamwriter.writerow(line_data_list)
with open('inputs', 'w',newline='', encoding='utf-8') as inputs:
    writer = tf.python_io.TFRecordWriter(inputs.name)
    for i in range(len(line_data_list)):
        token = line_data_list[i* sequence_length: (i+1) * sequence_length]
        label = line_data_list[(i* sequence_length)+1: (i+1) * sequence_length+1 ]
        ex = make_dataset(token, label)
        writer.write(ex.SerializeToString())
    writer.close()

### 데이터 만들고 불러오는 예제.

In [22]:
file_name_queue = tf.train.string_input_producer(['inputs'])
reader = tf.TFRecordReader()
_, data = reader.read(file_name_queue)

context_features = {
    "length": tf.FixedLenFeature([], dtype=tf.int64)
}
sequence_features = {
    "tokens": tf.FixedLenSequenceFeature([], dtype=tf.int64),
    "labels": tf.FixedLenSequenceFeature([], dtype=tf.int64)
}

context_parsed, sequence_parsed = tf.parse_single_sequence_example(serialized=data,
                                 context_features=context_features,
                                 sequence_features=sequence_features
                                )


In [23]:
context_parsed['length'] = tf.cast(context_parsed['length'], tf.int32)
batch_seq, batch_inputs = tf.contrib.training.bucket_by_sequence_length(context_parsed['length'],
                                                                            [sequence_parsed['tokens'], sequence_parsed['labels']],
                                                                            batch_size=128,
                                                                            bucket_boundaries=[5,10],
                                                                            dynamic_pad=True,
                                                                            capacity=1000,
                                                                            num_threads=1
                                                                           )

In [24]:
s = tf.Session()
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=s, coord=coord)
s.run(tf.global_variables_initializer())

length_,tokens_ = s.run([batch_seq, batch_inputs])
# print(length_, tokens_)
# print(tokens_['tokens'], tokens_['labels'])

### target은 input+1 의 인덱스로 하고, sequence는 실행시간에 정함.